In [1]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pltSave
import matplotlib.ticker as ticker
import seaborn as sns
import helper_fcns as hlp
import autoreload
import os
from scipy.stats import norm, wilcoxon
import statsmodels.api as sm
matplotlib.style.use('plt_style.mplstyle')
%matplotlib inline

Collect/consolidate data for GLM fit. Need trial-by-trial responses, split by contrast and spatial frequency. We will fit one model for each contrast, so we'll need the data consolidated over all spatial frequencies for a given contrast

In [2]:
dataDir = 'data/';
saveDir = dataDir + 'results/';
disp = 5;
isGeorgeson = 0;
savePlts = 0 ;
savePlts_logit = 1;

pmf_str = 'weibull';

weibull = lambda x, *params: 1 - np.exp(-np.power(x/params[0], params[1]));
true_pmf = lambda params, x: params[3] + (1-params[2]-params[3])*weibull(x, *params[0:2])
# true_pmf = lambda params, x: params[3] + (1-params[2]-params[3])*norm.cdf(x, *params[0:2])

if isGeorgeson:
    whichFiles = 'toAnalyzeD' + str(disp) + 'g.txt';
else:
    whichFiles = 'toAnalyzeD' + str(disp) + '.txt';
dataList = open(dataDir + whichFiles, 'r')
dFilesAll = dataList.readlines(-1)
dFilesAll = [x.strip('\n') for x in dFilesAll]
dataList.close()

In [3]:
if not isGeorgeson:
    if disp == 1:
        dFiles = dFilesAll[4:]; # ignore the first 4 sessions - early training
    elif disp == 3:
        dFiles = dFilesAll[1:];
    elif disp == 5:
        dFiles = dFilesAll[2:]; # ignore first 2
else:
    dFiles = dFilesAll;

In [4]:
sfIdx = 0;
conIdx = 1;
dispIdx = 2;
nInd = 3; # 3 indices per stimulus; add 3 to all of the above indices to get the equivalent value for the second stimulus
subjRespIdx = 2*nInd;
refIdx = subjRespIdx + 1; # which interval contained reference is after the subject's response in the data format

In [5]:
stim1 = 39; stim2 = 41; # subject responses (asked if first/left stimulus is of higher frequency than the second/right stimulus)

In [6]:
data = [];
for df in dFiles:
    currData = np.loadtxt(dataDir + df);
    if df == dFiles[0]:
        data = currData;
    else:
        data = np.concatenate((data, currData), axis = 0);

sfVals = np.union1d(np.unique(data[:, sfIdx]), np.unique(data[:, sfIdx+nInd]))
conVals = np.union1d(np.unique(data[:, conIdx]), np.unique(data[:, conIdx+nInd]))
nCons = len(conVals);
dispVals = np.union1d(np.unique(data[:, dispIdx]), np.unique(data[:, dispIdx+nInd]))

In [7]:
allResp = data[:, subjRespIdx];
if any((allResp != stim1) & (allResp != stim2)):
    print('Problem: response which is neither 1 or 2')

In [8]:
s1g = data[:, sfIdx] > data[:, sfIdx+nInd];
s2g = data[:, sfIdx] < data[:, sfIdx+nInd]

In [9]:
# find which one is the reference (as opposed to test) grating
ref = np.zeros(data.shape[0])
ref = data[:, refIdx]

# allows for possibility of more than one refSf...
refSF = np.union1d(np.unique(data[ref==1, sfIdx]), np.unique(data[ref==2, sfIdx+nInd]))

In [10]:
# did the subject perceive the test as higher SF?
testHF = np.zeros(data.shape[0]);
testHF[ref==1] = data[ref==1, subjRespIdx] == stim2;
testHF[ref==2] = data[ref==2, subjRespIdx] == stim1;

In [11]:
testCons = np.zeros(data.shape[0])
testCons[ref==1] = data[ref==1, nInd+conIdx];
testCons[ref==2] = data[ref==2, conIdx];

In [12]:
testSfs = np.zeros(data.shape[0])
testSfs[ref==1] = data[ref==1, nInd+sfIdx];
testSfs[ref==2] = data[ref==2, sfIdx];

In [13]:
checkz = np.zeros((len(testSfs), 1));
for i in range(len(sfVals)):
    checkz[testSfs == sfVals[i]] = 1;

In [14]:
# data analysis - number of trials/responses "test>ref sf", fraction "test>ref" split by sf/con
glmFits = [];
ptSf, nTr, nTestResp = (np.zeros((nCons, len(sfVals))) for _ in range(3));
for con in range(nCons):
    for sf in range(len(sfVals)):
        z = (testSfs == sfVals[sf]) & (testCons == conVals[con]) # get trials with the desired test sf/con
        nTr[con][sf] = sum(z); # how many trials in this configuration
        nTestResp[con][sf] = sum(testHF[z]) # how many trials in this configuration with response "test > ref sf"
        ptSf[con][sf] = nTestResp[con][sf] / nTr[con][sf]
    print('Con ' + str(conVals[con]) + ': ' + str(sum(nTr[con])) + ' trials')

Con 0.05: 192.0 trials
Con 0.1: 167.0 trials
Con 0.33: 190.0 trials
Con 1.0: 201.0 trials


In [ ]:
autoreload.reload(hlp);

In [ ]:
# optimize and bootstrap
nFits = 5; # don't think you need multistart, but build it in anyway
bootIter = 100; # set to 0 for no bootstrap
univ_params = 3; # number of "universal parameters", i.e. for all pmf
nParams = nCons + univ_params; # one mean for each contrast; overall slope, lapses
bootTestResps = np.zeros((nCons, len(nTestResp[0]), bootIter))
loss = np.zeros((nCons, 1)); # only one loss value...

opt = hlp.opt_pmf(sfVals, nTestResp, nTr, weibull, nFits);
# opt = hlp.opt_pmf(sfVals, nTestResp, nTr, norm.cdf, nFits);

params = np.zeros((nCons, univ_params+1)) # 3+1 params per PMF
for c in range(nCons): # now unpack...
    params[c, 0] = opt['x'][c];
    params[c, 1] = opt['x'][nCons+c];
    params[c, 2] = opt['x'][2*nCons+c];
    params[c, 3] = opt['x'][3*nCons+c];
    # expand dimensions of nTestResp so that pmf_loss knows there is only one contrast value to explore
    loss[c] = hlp.pmf_loss(sfVals, np.expand_dims(nTestResp[c], 0), np.expand_dims(nTr[c], 0),
                           norm.cdf, params[c, 0], params[c, 1], params[c, 2], params[c, 3]);
    
paramsBoot = np.zeros((nCons, univ_params+1, bootIter));
for iter in range(bootIter):
    bootTestResps[:, :, iter] = np.random.binomial(nTr.astype('int64'), ptSf)
    # expand dimensions of bootTestResps so that pmf_loss knows there is only one contrast value to explore
    optBoot = hlp.opt_pmf(sfVals, bootTestResps[:, :, iter], nTr, norm.cdf, 1)
    # unpack paramters
    for c in range(nCons):
        paramsBoot[c, 0, iter] = optBoot['x'][c];
        paramsBoot[c, 1, iter] = optBoot['x'][nCons+c];
        paramsBoot[c, 2, iter] = optBoot['x'][2*nCons+c];
        paramsBoot[c, 3, iter] = optBoot['x'][3*nCons+c];

In [ ]:
# compute from bootstrap values...
bootParamMean = paramsBoot.mean(-1);
bootParamStd = paramsBoot.std(-1);

# reshape number of trials by con/sf so we can get bootstrap estimates of subject's proportion responses "test>ref"
bootPtSf = bootTestResps / np.repeat(nTr.reshape((len(conVals), len(nTestResp[0]), 1)), 100, axis=-1); 
bootPtSfmean = bootPtSf.mean(-1);
bootPtSfstd = bootPtSf.std(-1);

### Plot of psychometric function and measured responses

In [ ]:
# now plot the PMF
whichCon = 1;
x = np.log2(np.arange(sfVals[0], sfVals[-1], 1e-2));
# true_pmf = lambda params: params[3] + (1-params[2]-params[3])*norm.cdf(x, *params[0:2])

curr_params = params[whichCon, :]
#sns.set_style("dark")

pts = plt.errorbar(np.log2(sfVals), ptSf[whichCon], yerr=bootPtSfstd[whichCon, :], clip_on=False, linestyle='none', marker='o');
refSFline = plt.plot(np.log2([refSF, refSF]), [0, 1], '--'); 
pse = plt.plot(np.log2([sfVals[0], sfVals[-1]]), [0.5, 0.5], '--')
pmf = plt.plot(x, true_pmf(curr_params, x), '-');
plt.ylim([0, 1]);
sfAsStr = [str(x) for x in np.round(sfVals, 2)];
plt.xticks(np.log2(sfVals), sfAsStr, rotation='45')
plt.xlabel('test SF (c/deg)');
plt.ylabel('proportion "test" higher SF');
plt.legend((pts[0], refSFline[0], pse[0], pmf[0]), ('subject responses', 'reference SF', 'PSE', 'fit pmf'));
plt.title('PMF for test contrast: ' + str(conVals[whichCon]) + ' -- ' + str(sum(nTr[whichCon])) + ' trials');

### Comparison of two contrasts

In [ ]:
con1 = 0; con2 = 3;
x = np.log2(np.arange(sfVals[0], sfVals[-1], 1e-2))
# true_pmf = lambda params: params[2] + (1-2*params[2])*norm.cdf(x, *params[0:2])

params1 = params[con1, :];
params2 = params[con2, :];

pts = plt.errorbar(np.log2(sfVals), ptSf[con1], yerr=bootPtSfstd[con1, :], clip_on=False, linestyle='none', marker='o');
pts2 = plt.errorbar(np.log2(sfVals), ptSf[con2], yerr=bootPtSfstd[con2, :], clip_on=False, linestyle='none', marker='o');
refSFline = plt.plot(np.log2([refSF, refSF]), [0, 1], '--'); 
pse = plt.plot(np.log2([sfVals[0], sfVals[-1]]), [0.5, 0.5], '--')
pmf = plt.plot(x, true_pmf(params1, x), '-');
pmf = plt.plot(x, true_pmf(params2, x), '-');
plt.ylim([0, 1]);
sfAsStr = [str(x) for x in np.round(sfVals, 2)];
plt.xticks(np.log2(sfVals), sfAsStr, rotation='45')
plt.xlabel('test SF (c/deg)');
plt.ylabel('proportion "test" higher SF');
plt.legend((pts[0], pts2[0], refSFline[0], pse[0], pmf[0]), (str(conVals[con1]), str(conVals[con2]), 'reference SF', 'PSE', 'fit pmf'));
plt.title('PMF for test contrasts: ' + str(conVals[con1]) + ' & ' + str(conVals[con2]));

### Plot all PMF

In [ ]:
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Helvetica']
rcParams['font.style'] = 'oblique'
#rcParams['font.size'] = 20
rcParams['pdf.fonttype'] = 3 # should be 42, but there are kerning issues
rcParams['ps.fonttype'] = 3 # should be 42, but there are kerning issues

In [ ]:
figPMF, allPMF = plt.subplots(1, 4, figsize=(20, 4));
# now plot the PMF

sns.set_style('white')
sns.despine(offset=10);

for c in range(nCons):
    whichCon = c;
    x = np.log2(np.arange(sfVals[0], sfVals[-1], 1e-2));
#     true_pmf = lambda params: params[2] + (1-2*params[2])*norm.cdf(x, *params[0:2])

    pts = allPMF[c].errorbar(np.log2(sfVals), ptSf[whichCon], yerr=bootPtSfstd[whichCon, :], clip_on=False, linestyle='none', marker='o');
    refSFline = allPMF[c].plot(np.log2([refSF, refSF]), [0, 1], '--'); 
    pse = allPMF[c].plot(np.log2([sfVals[0], sfVals[-1]]), [0.5, 0.5], '--')
    pmf = allPMF[c].plot(x, true_pmf(params[whichCon, :], x), '-');
    allPMF[c].set_ylim([0, 1]);
    allPMF[c].set_xticks(np.log2(sfVals))
    allPMF[c].tick_params(labelsize=15, width=2, length=15, which='major')
    allPMF[c].tick_params(labelsize=0, width=0, length=0, which='minor')
    sfAsStr = [str(x) for x in np.round(sfVals, 2)];
    allPMF[c].set_xticklabels(sfAsStr, rotation='45')
    allPMF[c].set_xlabel('test SF (c/deg)');
    allPMF[c].set_ylabel('prop "test" higher SF');
    allPMF[c].set_title('Contrast: ' + str(conVals[whichCon]) + ' -- ' + str(sum(nTr[whichCon])) + ' trials');
    if c == 0:
        allPMF[c].legend((pts[0], refSFline[0], pse[0], pmf[0]), ('subject responses', 'reference SF', 'PSE', 'fit pmf'), loc='upper left', fontsize=8);

figPMF.tight_layout()

What are the inferred PSEs for each contrast? How about the SF which is perceived at the reference SF?

In [ ]:
autoreload.reload(hlp);

In [ ]:
fit_pse = np.zeros((nCons, 1));
opts = [];
for c in range(nCons):
    fit_pse[c, 0], opts_curr = hlp.find_pse(true_pmf, params[c, :])
    opts.append(opts_curr);
    print('%.2f contrast: PSE is %.3f cpd' % (conVals[c], np.power(2, fit_pse[c, 0])))

### Plot of subject bias and sensitivity

In [ ]:
pse = np.zeros((len(conVals), 1));
evalSfs = np.arange(sfVals[0], sfVals[-1], 1e-3);
evalPmf = lambda params: params[2] + (1-2*params[2])*norm.cdf(evalSfs, *params[0:2]);
for con in range(len(conVals)):
    pse[con] = evalSfs[np.argmin(abs(evalPmf(params[con]) - 0.5))]

In [ ]:
# Statistics on PSE bootstrapped values...
pOfInt = 0; # PSE
outerInd = len(conVals);
pValPSE = np.nan*np.ones((outerInd, outerInd));
for c in range(outerInd):
    for ci in range(len(conVals) - c-1):
        
        z, pValPSE[c, c+ci+1] = wilcoxon(paramsBoot[c, pOfInt, :], paramsBoot[c+ci+1, pOfInt, :]); # 0 is PSE
        pValPSE[c+ci+1, c] = pValPSE[c, c+ci+1]

In [ ]:
print('Significant difference in bootstrap distributions of PSE? P value:')
upTri = np.triu(pValPSE);
np.set_printoptions(precision=4)
np.set_printoptions(suppress=False)
print(upTri)

In [ ]:
nParams = params.shape[0];
figParams, paramPlot = plt.subplots(1, nParams, figsize=(8*nParams, 4));

sns.set_style('white')
sns.despine(offset=10);

y_labels = ['SF (cpd)', 'sensitivity', 'lapse rate', 'lapse rate']
titles = ['Subject bias', 'Subject sensitivity', 'Lapse (ceiling)', 'Lapse (floor)']

for param_i in range(nParams):
    
    if param_i == 0:
        pse = paramPlot[param_i].errorbar(conVals, np.power(2, fit_pse[:, 0]), bootParamStd[:, param_i]);
    else:
        paramPlot[param_i].errorbar(conVals, params[:, param_i], bootParamStd[:, param_i]);
        
    paramPlot[param_i].set_xscale('log');
    paramPlot[param_i].set_xlabel('test contrast');
    paramPlot[param_i].set_ylabel(y_labels[param_i]);
    paramPlot[param_i].set_title(titles[param_i]);
    paramPlot[param_i].tick_params(labelsize=15, width=2, length=15, which='major')
    paramPlot[param_i].set_xticks(conVals);
    conAsStr = [str(x) for x in conVals];
    paramPlot[param_i].set_xticklabels(conAsStr);

# paramPlot[2].text(0.5, 0.55, 'Lapse high: ' + str(np.round(params[0, 2], 3)) + ' +/- ' + 
#                    str(np.round(bootParamStd[0, 1], 3)), horizontalalignment = 'center', verticalalignment = 'center');
# paramPlot[2].text(0.5, 0.45, 'Lapse low: ' + str(np.round(params[0, 3], 3)) + ' +/- ' + 
#                   str(np.round(bootParamStd[0, 2], 3)), horizontalalignment = 'center', verticalalignment = 'center');
# paramPlot[2].set_title('Other parameters');

In [ ]:
if savePlts:
    allFigs = [figPMF, figParams];
    if isGeorgeson:
        saveName = "results_%sD%dg.pdf" % (pmf_str, disp)
    else:
        saveName = "results_%sD%d.pdf" % (pmf_str, disp)
    pdf = pltSave.PdfPages(str(saveDir + saveName))
    for fig in range(len(allFigs)): ## will open an empty extra figure :(
        pdf.savefig(allFigs[fig], bbox_inches="tight")
    pdf.close()

### Now do it with a GLM (logit link)

In [ ]:
sfsPlot = np.logspace(np.log10(sfVals[0]), np.log10(sfVals[-1]), 101)
glmFits = dict();

for c in range(nCons):
    curr_tr = testCons == conVals[c]; # get trials with the desired test sf/con
    resps = testHF[curr_tr];
    sfs = testSfs[curr_tr];

    curr_fit = hlp.fit_pmf_logit(resps, sfs);
    glmFits[c] = dict();
    glmFits[c]['opt'] = curr_fit;
    pse, pse_se, curve = hlp.get_logit_results(curr_fit, eval_sfs=sfsPlot);
    glmFits[c]['pse'] = pse;
    glmFits[c]['pse_se'] = pse_se;
    glmFits[c]['curve'] = curve;

In [ ]:
figPMF, allPMF = plt.subplots(1, 4, figsize=(20, 4));
# now plot the PMF

sns.set_style('white')
sns.despine(offset=10);

for c in range(nCons):
    whichCon = c;
    x = np.log2(sfsPlot);

    pts = allPMF[c].errorbar(np.log2(sfVals), ptSf[whichCon], yerr=bootPtSfstd[whichCon, :], clip_on=False, linestyle='none', marker='o');
    refSFline = allPMF[c].plot(np.log2([refSF, refSF]), [0, 1], '--'); 
    pse = allPMF[c].plot(np.log2([sfVals[0], sfVals[-1]]), [0.5, 0.5], '--')
    pmf = allPMF[c].plot(x, glmFits[c]['curve'], '-');
    allPMF[c].set_ylim([0, 1]);
    allPMF[c].set_xticks(np.log2(sfVals))
    allPMF[c].tick_params(labelsize=15, width=2, length=15, which='major')
    allPMF[c].tick_params(labelsize=0, width=0, length=0, which='minor')
    sfAsStr = [str(x) for x in np.round(sfVals, 2)];
    allPMF[c].set_xticklabels(sfAsStr, rotation='45')
    allPMF[c].set_xlabel('test SF (c/deg)');
    allPMF[c].set_ylabel('prop "test" higher SF');
    allPMF[c].set_title('Contrast: ' + str(conVals[whichCon]) + ' -- ' + str(sum(nTr[whichCon])) + ' trials');
    if c == 0:
        allPMF[c].legend((pts[0], refSFline[0], pse[0], pmf[0]), ('subject responses', 'reference SF', 'PSE', 'fit pmf'), loc='upper left', fontsize=8);

figPMF.tight_layout()

In [ ]:
glmFits.keys()

In [ ]:
figParams, paramPlot = plt.subplots(1, 1);
sns.set_style('white')
sns.despine(offset=10);

pse = [glmFits[x]['pse'] for x in glmFits.keys()];
pse_se = [glmFits[x]['pse_se'] for x in glmFits.keys()];

paramPlot.errorbar(conVals, pse, pse_se)

paramPlot.set_xscale('log');
paramPlot.set_xlabel('test contrast');
paramPlot.set_ylabel('sf (cpd)');
paramPlot.set_title('Point of subjective equality');
paramPlot.tick_params(labelsize=15, width=2, length=15, which='major')
paramPlot.set_xticks(conVals);
conAsStr = [str(x) for x in conVals];
paramPlot.set_xticklabels(conAsStr);

# paramPlot[2].text(0.5, 0.55, 'Lapse high: ' + str(np.round(params[0, 2], 3)) + ' +/- ' + 
#                    str(np.round(bootParamStd[0, 1], 3)), horizontalalignment = 'center', verticalalignment = 'center');
# paramPlot[2].text(0.5, 0.45, 'Lapse low: ' + str(np.round(params[0, 3], 3)) + ' +/- ' + 
#                   str(np.round(bootParamStd[0, 2], 3)), horizontalalignment = 'center', verticalalignment = 'center');
# paramPlot[2].set_title('Other parameters');

In [ ]:
logit_str = 'logit'
if savePlts_logit:
    allFigs = [figPMF, figParams];
    if isGeorgeson:
        saveName = "results_%sD%dg.pdf" % (logit_str, disp)
    else:
        saveName = "results_%sD%d.pdf" % (logit_str, disp)
    pdf = pltSave.PdfPages(str(saveDir + saveName))
    for fig in range(len(allFigs)):
        pdf.savefig(allFigs[fig], bbox_inches="tight")
    pdf.close()